In [1]:
import sys
import glob
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
sys.path.append(r"C:\Users\scanimage\Documents\JJM\post_cnmfe_analysis")
import dlc_utils as dlc
import os
import datetime
import math
import time 
from tqdm import tqdm
from matplotlib.dates import DateFormatter, date2num

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

C:\Users\scanimage\AppData\Local\Temp\ipykernel_15120\912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
data_folder = r"F:\\JJM\\behavCam_videos_for_analysis\\v4_fenobaminjections\\"
ezTrack_location_files = glob.glob(data_folder+ r"*\\*LocationOutput.csv")

In [4]:
#ezTrack_location_files = glob.glob(data_folder+ r"\\*GRIN*\\" + '*LocationOutput.csv')

In [5]:
len(ezTrack_location_files)

17

In [6]:
ezTrack_location_files[0:17]

['F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.14_00_14_00_13\\DIO_r2.14_00_14_00_13combined_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.14_00_17_25_26\\DIO_r2.14_00_17_25_26_combined_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.14_01_14_30_28\\DIO_r2.14_01_14_30_28combined_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.14_01_17_40_46\\DIO_r2.14_01_17_40_46_combined_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.14_02_18_11_01\\DIO_r2.14_02_18_11_01_combined_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.19_13_45_01\\concactenated_behavCam00_behavCam27_LocationOutput.csv',
 'F:\\\\JJM\\\\behavCam_videos_for_analysis\\\\v4_fenobaminjections\\DIO_r2.19_14_15_20\\concactenated_behavCam00_behavCam27_LocationOut

In [7]:
def alignBinVelocity(sessionPath, savePath, downSampleFactorForVelocity, medianFilter, velocityBins):

    #'\\'.join(sessionPath.split(os.sep)[:-1])+'\\timestamp.dat'
    print(sessionPath.split(os.sep)[:-1])
    # load eZ track output and behavior camera timestamps from miniscope software 
    ezTrackOutput = pd.read_csv(sessionPath)
    timestampfile = pd.read_table('\\'.join(sessionPath.split(os.sep)[:-1])+'\\timestamps.csv', delimiter=',')
    miniscope_timestampfile = pd.read_table('\\'.join(sessionPath.split(os.sep)[:-1])+'\\timestampsMiniscope.csv', delimiter=',')
    
    frame_rate=15
    timestampfile_td = timestampfile.set_index(pd.to_timedelta(np.linspace(0, (len(timestampfile)-1)*(1/frame_rate), len(timestampfile)), unit='s'), drop=False)
    behavCamTimeStampsDownsampled = timestampfile_td.resample('.2S').bfill()
    behavCamTimeStampsMedian = behavCamTimeStampsDownsampled.resample('1S').median()
    behavCamTimeStampsResampled = behavCamTimeStampsMedian.resample('.2S').bfill()
    
    frame_rate=20
    miniscopetimestamp_td = miniscope_timestampfile.set_index(pd.to_timedelta(np.linspace(0, (len(miniscope_timestampfile)-1)*(1/frame_rate), len(miniscope_timestampfile)), unit='s'), drop=False)
    miniscopeTimeStampsDownsampled = miniscopetimestamp_td.resample('.2S').bfill()
    miniscopeTimeStampsMedian = miniscopeTimeStampsDownsampled.resample('1S').median()
    miniscopeTimeStampsResampled = miniscopeTimeStampsMedian.resample('.2S').bfill()
    
    
    behavCam_frames = []
    sys_clock_behavCam = []
    #create "key" for aligning miniscope frames to timestamp file
    #then create behavior TD and align
    for msCam_frame in tqdm(range(0, len(miniscopeTimeStampsResampled['Frame Number']))):
        #get sys clock time of each miniscope recorded frame
        #sys_clock_msCam = time_stamps['sysClock'].loc[msCam_frame]
        #find behav cam frame closest to sys clock time of ms frame
        behavCam_frame = list(behavCamTimeStampsResampled.iloc[(behavCamTimeStampsResampled['Time Stamp (ms)']-miniscopeTimeStampsResampled['Time Stamp (ms)'].iloc[msCam_frame]).abs().argsort()[:1]].index)[0]
        behavCam_frames.append(behavCam_frame)
        sys_clock_behavCam.append(behavCamTimeStampsResampled.loc[behavCam_frame]['Time Stamp (ms)'])

    behavCamIdxToAlign = [behavCamTimeStampsResampled.index.get_loc(idx) for idx in behavCam_frames]
    #ezTrackOutput

    # separate behavCam time stamps
    behavCamTimeStamps = timestampfile['Time Stamp (ms)']
    #behavCamTimeStamps['frameNum'][0:len(ezTrackOutput)]
    # get times of frames
    behavCamTimes = behavCamTimeStamps
    # set 1st frame to 0 
    behavCamTimes[1] = 0 
    # reset index to 0 
    #behavCamTimes = behavCamTimes.reset_index()

    # delta T between frames for analyzed videos 
    behavCamFrameTimeDelta = behavCamTimes.diff()[0:len(ezTrackOutput)]
    
    # get velocity from frame times 
    velocity = abs(ezTrackOutput['Distance_cm'].diff())/(behavCamFrameTimeDelta/1000)

    # data frame of mouse velocity 
    veloctiyDataFrame = pd.concat([ezTrackOutput[['Frame', 'X', 'Y', 'Distance_px', 'Distance_cm']], 
                               behavCamTimes[0:len(ezTrackOutput)],
                               pd.DataFrame(velocity, columns=['Velocity'])], axis=1)
    
    #calculate frame rate 
    frame_rate = int(1000/veloctiyDataFrame['Time Stamp (ms)'].diff().mean())

    # create time delta for behavior index
    # behavCam for v3 should record at 30 fps, can verify in timestamp file, check this is set correctly in concactenated videos
    veloctiyDataFrame = veloctiyDataFrame.set_index(pd.to_timedelta(np.linspace(0, (len(veloctiyDataFrame)-1)*(1/frame_rate), len(veloctiyDataFrame)), unit='s'))
    veloctiyDataFrame

    #len(veloctiyDataFrame)

    ## smooth velocity by downsampling to 5Hz to calculate distance travelled 
    downsampledDataFrame = veloctiyDataFrame.resample('.2S').mean()[['X', 'Y']]
    # calculate new euclidean distance based on mean values here 


    cmPerPixel = veloctiyDataFrame['Distance_cm'][1]/veloctiyDataFrame['Distance_px'][1]

    print(cmPerPixel)

    def euclideanDistance(row):
        return(math.sqrt(row['X']**2+row['Y']**2))

    def smoothedVelocity(row, cmPerPixel):
        #euclidean distance divided by sampling rate, multiplied by cm/pixel
        return((math.sqrt(row['X']**2+row['Y']**2)/.2)*cmPerPixel)
    
    downsampledDataFrame['smoothedVelocity'] = downsampledDataFrame.diff().apply(lambda row : smoothedVelocity(row, cmPerPixel), axis=1)

    downsampledDataFrame

    #downsample once more to 2S as a "median filter"
    downsampledMedian = downsampledDataFrame.resample(medianFilter).median()

    #resample to 5Hz for alignment with imaging data
    resampledToAlign = downsampledMedian.resample('.2S').bfill()
    ## remove frames here to align with miniscope df 
    velocity_resampled = resampledToAlign.iloc[behavCamIdxToAlign]
    
    
    ## save speed bins
    #velocity_resampled[(velocity_resampled['smoothedVelocity']>0) & (velocity_resampled['smoothedVelocity']<0.5)]

    # for "rest" analysis maybe further split < 0.5 cm/sec bins e.g. (0, 0.25) (0, 0.5)

    # define velocity bins and save output to csv files for clustering analysis
    
    v_bin_dict = {}

    #save data frames of bin indicies in dictionary
    for bin_idx in range(len(velocityBins)):
        v_bin_dict[bin_idx]=pd.DataFrame([velocity_resampled.index.get_loc(velocity_resampled[(velocity_resampled['smoothedVelocity']>velocityBins[bin_idx][0]) & (velocity_resampled['smoothedVelocity']<velocityBins[bin_idx][1])].index[i]) 
         for i in range(len(velocity_resampled[(velocity_resampled['smoothedVelocity']>velocityBins[bin_idx][0]) & (velocity_resampled['smoothedVelocity']<velocityBins[bin_idx][1])].index))], columns=['Indicies'])
    #save to csv files
    for bin_idx in range(len(velocityBins)):
        v_bin_dict[bin_idx].to_csv(savePath+os.path.split(sessionPath)[0].split('\\')[-1]+'_velocityBin'+str(bin_idx)+'.csv')
    
    return(v_bin_dict)


In [8]:
savePath = r'F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\clustering_analysis\\frames_subset_'+time.strftime("%a_%d_%b_%Y_%H_%M_%S")+'\\'
os.mkdir(savePath)
downSampleFactorForVelocity = '.2S'
medianFilter = '1S'
vBins = [(0, 0.75), (0.75, 1.5), (1.5, 2.5), (2.5, 6), (6, 10), (10, math.inf)]
batch_output = {}
for sessionPath in  ezTrack_location_files[0:16]:
    v_bin_dict = alignBinVelocity(sessionPath, savePath, downSampleFactorForVelocity, medianFilter, vBins)
    batch_output[os.path.split(sessionPath)[0].split('\\')[-1]]=v_bin_dict
    
params = pd.DataFrame({'downSampleFactorForVelocity': downSampleFactorForVelocity, 'medianFilter': medianFilter}, index=['params'])
params.to_csv(savePath+'\\'+'params.csv')
vBins = pd.DataFrame(vBins)
vBins.to_csv(savePath+'\\'+'vBins.csv')

['F:', '', 'JJM', '', 'behavCam_videos_for_analysis', '', 'v4_fenobaminjections', 'DIO_r2.14_00_14_00_13']


100%|████████████████████████████████████████████████████████████████████████████| 8921/8921 [00:06<00:00, 1302.89it/s]


0.12252631327288699
['F:', '', 'JJM', '', 'behavCam_videos_for_analysis', '', 'v4_fenobaminjections', 'DIO_r2.14_00_17_25_26']


100%|████████████████████████████████████████████████████████████████████████████| 4466/4466 [00:03<00:00, 1459.66it/s]


0.11795811623765394
['F:', '', 'JJM', '', 'behavCam_videos_for_analysis', '', 'v4_fenobaminjections', 'DIO_r2.14_01_14_30_28']


...

0.12078966983174881
['F:', '', 'JJM', '', 'behavCam_videos_for_analysis', '', 'v4_fenobaminjections', 'DIO_r2.8_16_30_39']


100%|████████████████████████████████████████████████████████████████████████████| 8896/8896 [00:06<00:00, 1285.52it/s]


0.12156324401893985
['F:', '', 'JJM', '', 'behavCam_videos_for_analysis', '', 'v4_fenobaminjections', 'DIO_r2.8_16_45_11']


100%|████████████████████████████████████████████████████████████████████████████| 8921/8921 [00:06<00:00, 1305.59it/s]


0.1268858800773694


In [ ]:
## velocity dataframe plot unsmoothed
xRangeFrames=(0,58000)
y = veloctiyDataFrame['Velocity'][xRangeFrames[0]:xRangeFrames[1]]
#recording_start_time = datetime.datetime.fromtimestamp(abs(timestampfile[timestampfile['camNum'] == 1]['sysClock'][1]))
#plot x axis time
recording_start_time = datetime.datetime(1970, 1, 1, hour=0, minute=0, second=0, microsecond=0)
x = [recording_start_time+veloctiyDataFrame.index[i] for i in range(len(veloctiyDataFrame.index))][xRangeFrames[0]:xRangeFrames[1]]

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
fig.subplots_adjust(bottom=0.2)

myFmt = DateFormatter("%M:%S:%ms")
ax1.xaxis.set_major_formatter(myFmt)
ax1.plot(x, y)
plt.gcf().autofmt_xdate()

# Move twinned axis ticks and label from top to bottom
ax2.set_xlim(0, len(veloctiyDataFrame))
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.25))

plt.show()

In [ ]:
len(velocity_resampled)

In [ ]:
## plot smoothed frames 
velocity_resampled = downsampledMedian.resample('.2S').bfill()
xRangeFrames=(0,len(velocity_resampled))
columnToPlot = 'smoothedVelocity'
y = velocity_resampled[columnToPlot][xRangeFrames[0]:xRangeFrames[1]]
#recording_start_time = datetime.datetime.fromtimestamp(abs(timestampfile[timestampfile['camNum'] == 1]['sysClock'][1]))
#plot x axis time
recording_start_time = datetime.datetime(1970, 1, 1, hour=0, minute=0, second=0, microsecond=0)
x = [recording_start_time+velocity_resampled.index[i] for i in range(len(velocity_resampled.index))][xRangeFrames[0]:xRangeFrames[1]]

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
fig.subplots_adjust(bottom=0.2)

myFmt = DateFormatter("%M:%S:%ms")
ax1.xaxis.set_major_formatter(myFmt)
ax1.plot(x, y)
plt.gcf().autofmt_xdate()

# Move twinned axis ticks and label from top to bottom
ax2.set_xlim(0, len(velocity_resampled))
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.25))

plt.show()

In [ ]:
indiciesRestbin3

In [ ]:
#move to quest
#res_files=8f796c9e-f5c8-11e5-9842-22000b9da45e
#quest=d5990400-6d04-11e5-ba46-22000b92c6ec
#res_files_base_dir = '8f796c9e-f5c8-11e5-9842-22000b9da45e:/rdss/jma819/fsmresfiles/Projects/JJM/MiniscopeMovies/scope_data/'
#quest_dir = 'd5990400-6d04-11e5-ba46-22000b92c6ec:/projects/b1118/behaviorvideos/timestampfiles/'

In [ ]:
#for session in list(time_stamp_file_locations.keys()):
#    transfer_info = res_files_base_dir+time_stamp_file_locations[session]+' '+quest_dir+session+'_timestamp.dat'
#    !eval "globus transfer" $transfer_info

In [ ]:
##frame by frame alignment to miniscope camera

sessions = [f_path.split('/')[-1].rstrip('_raw_trace.csv') for f_path in mm_raw_traces]

sessions = ['GRIN039_H14_M8_S53', 'GRIN026_H16_M35_S34', 'GRIN032_H16_M49_S22']

for session in sessions:
    print(session)
    timestampfile = '/projects/b1118/behaviorvideos/timestampfiles/'+session+'_timestamp.dat'
    frame_clock_df = pd.read_table(timestampfile)
    output_directory = '/projects/b1118/miniscope/analysis/compiled_data/mm_tracking_output/'

    Cam0_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0]  
    Cam1_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1]
    #beavCam is usually faster framerate 
    if Cam1_timestamps['sysClock'].iloc[1:].diff().mean()<Cam0_timestamps['sysClock'].iloc[1:].diff().mean():
        msCam_camnum=0
        behavCam_camnum=1
    elif Cam0_timestamps['sysClock'].iloc[1:].diff().mean()<Cam1_timestamps['sysClock'].iloc[1:].diff().mean():
        msCam_camnum=1
        behavCam_camnum=0

    raw_trace = pd.read_csv(data_folder+session+'_raw_trace.csv', names=['velocity'])
    raw_trace_filtered = pd.read_csv(data_folder+session+'_raw_trace_median_filter.csv', names=['velocity(20_Hz_filtered)'])
    speed_trace = pd.read_csv(data_folder+session+'_speedtrace.csv', names=['speed_trace(velocity_5Hz_filtered)'])
    raw_traces = pd.concat([raw_trace, raw_trace_filtered], axis=1)

    # align the raw traces for comparison

    mm_tracking_aligned = dlc.downsample_mmtracking(raw_traces, timestampfile, msCam_camnum, behavCam_camnum)

    # convert to timedelta for downsampling 
    aligned_td = mm_tracking_aligned.set_index(pd.to_timedelta(np.linspace(0, mm_tracking_aligned['sys_clock_behavCam'].values[-1]/1000, len(mm_tracking_aligned)), unit='s'), drop=False)
    aligned_td.to_csv(output_directory+session+'_mm_tracking_foranalysis_unsmoothed'+'.csv')